# dependencies

In [1]:
import sys
sys.path.insert(0, '../')  # Adjust the path as necessary
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from inverse_canopy import InverseCanopy

2024-02-11 00:19:42.451225: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# **Model Inputs**

In [ ]:
from inverse_canopy import InverseCanopy
import tensorflow as tf
import numpy as np

tunable = {
 'num_samples': 10000,    # number of monte carlo samples
 'learning_rate': 0.1,    # the gradient update rate
 'dtype': tf.float64,     # use 64-bit floats
 'epsilon': 1e-30,        # useful for avoiding log(0 + epsilon) type errors
 'max_steps': 5000,       # maximum steps, regardless of convergence
 'patience': 50,          # number of steps to wait before early stopping if the loss does not improve   
}

conditional_events = {
    'names': ['LCDL', 'PKRU', 'DHRS', 'DHRL'],
    'bounds': {
        'mean': {
            'min': 1e-14,
            'max': 1.00,
        },
        'std': {
            'min': 1e-10,
            'max': 1e8,
        },
     },
    'initial': {
       'mean': 5e-1,
       'std': 1e8,
    }
}

end_states = {
    'LCDL-1': {
        'sequence': [1, 0, 0, 0],
        'probability': 1.1e-8,
    },
    'LCDL-2': {
        'sequence': [1, 0, 0, 1],
        'probability': 1.00e-11,
    },
    'LCDL-3': {
        'sequence': [1, 0, 1, np.nan],
        'probability': 1.00e-11,
    },
    'LCDL-4': {
        'sequence': [1, 1, np.nan, np.nan],
        'probability': 1.00e-11,
    },
    'LCDL-0': {
        'sequence': [0, np.nan, np.nan, np.nan],
        'probability': 1.0 - 1.1e-8 - 1.00e-11 - 1.00e-11 - 1.00e-11, # one minus all the other end-states
    },
}

model = InverseCanopy(conditional_events, end_states, tunable)
model.fit(steps=tunable['max_steps'], patience=tunable['patience'], learning_rate=tunable['learning_rate'])
model.summarize(show_plot=True, show_metrics=False)

In [ ]:
tunable = {
 'num_samples': 1000,     # number of monte carlo samples
 'learning_rate': 0.1,    # the gradient update rate
 'dtype': tf.float64,     # use 64-bit floats
 'epsilon': 1e-30,        # useful for avoiding log(0 + epsilon) type errors
 'max_steps': 5000,       # maximum steps, regardless of convergence
 'patience': 50,          # number of steps to wait before stopping, if the loss does not improve
}

conditional_events = {
    'names': ['SDFR      ', 'LMFD      ', 'RFIR      ', 'LLRF      ', 'SSSD|~LLRF', 'SSSD|LLRF ', 'SYSO|~LLRF', 'SYSO|LLRF '],  # Assuming names based on the sequences
    'bounds': {
        'mean': {
            'min': 1e-14,
            'max': 1.00,
        },
        'std': {
            'min': 1e-10,
            'max': 1e8,
        },
     },
    'initial': {
       'mean': 5e-1,
       'std': 1e8,
    }
}

end_states = {
    'SDFR-0': {
        'sequence': [0, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
        'probability': 1 - 4.2e-3 - 4.2e-5 - 5.8e-9 - 2.1e-5 - 1.9e-5 - 2.1e-6 - 1.7e-8 - 4.3e-9,
    },
    'SDFR-1': {
        'sequence': [1, 0, 0, 0, 0, np.nan, np.nan, np.nan],
        'probability': 4.2e-3,
    },
    'SDFR-2': {
        'sequence': [1, 0, 0, 0, 1, np.nan, 0, np.nan]  ,
        'probability': 4.2e-5,
    },
    'SDFR-3': {
        'sequence': [1, 0, 0, 0, 1, np.nan, 1, np.nan],
        'probability': 5.8e-9,
    },
    'SDFR-4': {
        'sequence': [1, 0, 0, 1, np.nan, 0, np.nan, np.nan],
        'probability': 2.1e-5,
    },
    'SDFR-5': {
        'sequence': [1, 0, 0, 1, np.nan, 1, np.nan, 0],
        'probability': 1.9e-5,
    },
    'SDFR-6': {
        'sequence': [1, 0, 0, 1, np.nan, 1, np.nan, 1],
        'probability': 2.1e-6,
    },
    'SDFR-7': {
        'sequence': [1, 0, 1, np.nan, np.nan, np.nan, np.nan, np.nan],
        'probability': 1.7e-8,  
    },
    'SDFR-8': {
        'sequence': [1, 1, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
        'probability': 4.3e-9,  
    },
}

model = InverseCanopy(conditional_events, end_states, tunable)
model.fit(steps=tunable['max_steps'], patience=tunable['patience'], learning_rate=tunable['learning_rate'])
model.summarize(show_plot=True, show_metrics=False)